<a href="https://colab.research.google.com/github/Ashish2Parimi/Telugu_Headline_Generator/blob/main/Sub%20word%20token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# `BertTokenizer.detokenize` is not in `tf-text` stable yet (currently 2.4.3).
!pip install -q tensorflow_text_nightly
# tf-text-nightly resquires tf-nightly
!pip install -q tf-nightly

     |████████████████████████████████| 4.3MB 8.7MB/s 
     |████████████████████████████████| 455.5MB 38kB/s 
     |████████████████████████████████| 4.0MB 36.7MB/s 
     |████████████████████████████████| 6.1MB 32.8MB/s 
     |████████████████████████████████| 471kB 53.2MB/s 
     |████████████████████████████████| 1.3MB 26.9MB/s 
     |████████████████████████████████| 4.2MB 45.3MB/s 
     |████████████████████████████████| 3.9MB 42.4MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.


In [ ]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]


In [ ]:
tokenizer = text.BertTokenizer('vocab.txt', **bert_tokenizer_params)


In [ ]:
import pandas as pd
data = pd.read_csv('/content/train_telugu_news.csv',encoding ='UTF-8')

data = data.drop(['SNo','date','topic'],axis = 1)
data.dropna(axis = 0,inplace = True)

# data = data.applymap(lambda x:x.encode(encoding='UTF-8'))

maxlen = 1024
max_features = 10
def format_data(data):
   
    
    # Tokenize text
    data = data.applymap(lambda x: str(x) if isinstance(x, int) or isinstance(x, float) else x)
    data['heading'] = data['heading'].str.replace('\d+', '')
    # data['heading'] = data['heading'].apply(lambda x:vect_dvect.vect_2(x,False,maxlen,max_features))

    data['body'] = data['body'].str.replace('\d+', '')
    # data['body'] = data['body'].apply(lambda x:vect_dvect.vect_2(x,True,maxlen,max_features))

    # data = data.applymap(lambda x:vect(x))
    X = data['body'].tolist()
    Y = data['heading'].tolist()
    
    return X, Y

X, Y = format_data(data)
target = data.pop('heading')
data = data.pop('body')


train_examples = tf.data.Dataset.from_tensor_slices((data.values, target.values))

In [ ]:
for pt, en in train_examples.take(1):
  print("Portuguese: ", pt.numpy().decode('utf-8'))
  print("English:   ", en.numpy().decode('utf-8'))

Portuguese:  భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ వ్యవహారాలపై భారత రిజర్వు బ్యాంకు (ఆర్‌బిఐ) ఒక కన్నేసింది. ఐడిబిఐ బ్యాంకును ఆర్‌బిఐ వాచ్‌లి్‌స్టలో చేర్చినట్టుగా తెలిసింది. నికర మొండిపద్దుల మొత్తం 6 శాతం దాటడం, వరసగా రెండేళ్ల పాటు నష్టాలను  ప్రకటించడం, కాపిటల్‌ అడెక్వసీ నిర్దేశిత ప్రమాణాల కంటే తగ్గడం... ఈ సందర్భాల్లో బ్యాంకులను ఆర్‌బిఐ వాచ్‌ లిస్ట్‌లో చేరుస్తుంది. తమ బ్యాంకుకు సంబంధించి ఆర్‌బిఐ ప్రాంప్ట్‌ కరెక్టీవ్‌ యాక్షన్‌ (పిసిఎ) చేపట్టినట్టు ఐడిబిఐ వెల్లడించింది. గతంలో ఓవర్సీస్‌ బ్యాంక్‌, యునైటెడ్‌ బ్యాంక్‌కు సంబంధించి  కూడా ఆర్‌బిఐ పిసిఎ చేపట్టింది. పిసిఎ వల్ల బ్యాంకుపై ఆర్థికంగా భారం పడదనీ, దీనివల్ల అంతర్గత నియంత్రణలు మెరుగుపడటంతో పాటు, కార్యకలాపాలు మెరుగవుతాయని ఐడిబిఐ వివరించింది. ఆర్‌బిఐ ఆదేశాలకు అనుగుణంగా ఇప్పుడు మూలధనాన్ని పొదుపుగా వినియోగించడంపై  ఐడిబిఐ దృష్టి సారించాల్సి ఉంటుంది.ఇందులో భాగంగా పరపతిపై నియంత్రణలు విధించడం, నియామకాలను నిలిపేయడం, కొత్త పెట్టుబడి పథకాలను పక్కన బెట్టడంవంటి చర్యలు తీసుకోవాల్సి ఉంటుం ది. బ్యాంకు సరైన రీతిలో స్పందించని పక్షంలో మరో బలమైన బ్య

In [ ]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

In [ ]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy().decode('utf-8'))

ఐడిబిఐపై ఆర్‌బిఐ నజర్‌
బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ
కీలక వికెట్ తీసిన జడేజా..


In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab


In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [ ]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

TypeError: ignored

In [ ]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '#', '%', '&', "'"]
['న', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల']
['##చచు', '##వస', 'ఎననకలు', 'మందక', 'సవల', 'ఎవరు', 'నటుడు', 'భరగ', 'వధనం', 'ఆసకత']
['##౦', '##౧', '##౨', '##౩', '##౪', '##–', '##‘', '##’', '##“', '##”']


In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [ ]:
write_vocab_file('pt_vocab.txt', pt_vocab)


In [ ]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 38 s, sys: 238 ms, total: 38.3 s
Wall time: 38.3 s


In [ ]:
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '%', "'", '(', ')']
['‘', '’', '##ల', '##న', '##కు', '##ప', '##త', '##క', '##ు', '##ం']
['ఆఫరలు', 'ఆశలు', 'ఈస', 'ఊరట', 'కటమరయుడు', 'కడుకు', 'కద', 'కవరటరసల', 'కషటలు', 'గపప']
['##ఙ', '##ఛ', '##ఝ', '##ఞ', '##ఢ', '##ృ', '##ౄ', '##–', '##‘', '##’']


In [ ]:
write_vocab_file('en_vocab.txt', en_vocab)


In [ ]:
pt_tokenizer = text.BertTokenizer('pt_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('en_vocab.txt', **bert_tokenizer_params)


In [53]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy().decode('utf-8'))

ఐడిబిఐపై ఆర్‌బిఐ నజర్‌
బ్యాంకింగ్‌ చీఫ్‌లతో నేడు జైట్లీ భేటీ
కీలక వికెట్ తీసిన జడేజా..


In [54]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
  print(ex)

[57, 116, 173, 453, 105, 572, 80, 325]
[1780, 420, 232, 316, 431, 404]
[287, 1154, 1929, 1763, 13, 13]


In [55]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(en_vocab, token_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\xe0\xb0\x90 ##\xe0\xb0\xa1 ##\xe0\xb0\xac ##\xe0\xb0\x90 ##\xe0\xb0\xaa \xe0\xb0\x86\xe0\xb0\xb0\xe0\xb0\xac\xe0\xb0\x90 \xe0\xb0\xa8 ##\xe0\xb0\x9c\xe0\xb0\xb0',
       b'\xe0\xb0\xac\xe0\xb0\xaf\xe0\xb0\x82\xe0\xb0\x95\xe0\xb0\x82\xe0\xb0\x97 \xe0\xb0\x9a\xe0\xb0\xab ##\xe0\xb0\xb2\xe0\xb0\xa4 \xe0\xb0\xa8\xe0\xb0\xa1\xe0\xb1\x81 \xe0\xb0\x9c\xe0\xb0\x9f\xe0\xb0\xb2 \xe0\xb0\xad\xe0\xb0\x9f',
       b'\xe0\xb0\x95\xe0\xb0\xb2\xe0\xb0\x95 \xe0\xb0\xb5\xe0\xb0\x95\xe0\xb0\x9f \xe0\xb0\xa4\xe0\xb0\xb8\xe0\xb0\xa8 \xe0\xb0\x9c\xe0\xb0\xa1\xe0\xb0\x9c . .'],
      dtype=object)>

In [56]:
words = en_tokenizer.detokenize(token_batch)
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\xe0\xb0\x90\xe0\xb0\xa1\xe0\xb0\xac\xe0\xb0\x90\xe0\xb0\xaa \xe0\xb0\x86\xe0\xb0\xb0\xe0\xb0\xac\xe0\xb0\x90 \xe0\xb0\xa8\xe0\xb0\x9c\xe0\xb0\xb0',
       b'\xe0\xb0\xac\xe0\xb0\xaf\xe0\xb0\x82\xe0\xb0\x95\xe0\xb0\x82\xe0\xb0\x97 \xe0\xb0\x9a\xe0\xb0\xab\xe0\xb0\xb2\xe0\xb0\xa4 \xe0\xb0\xa8\xe0\xb0\xa1\xe0\xb1\x81 \xe0\xb0\x9c\xe0\xb0\x9f\xe0\xb0\xb2 \xe0\xb0\xad\xe0\xb0\x9f',
       b'\xe0\xb0\x95\xe0\xb0\xb2\xe0\xb0\x95 \xe0\xb0\xb5\xe0\xb0\x95\xe0\xb0\x9f \xe0\xb0\xa4\xe0\xb0\xb8\xe0\xb0\xa8 \xe0\xb0\x9c\xe0\xb0\xa1\xe0\xb0\x9c . .'],
      dtype=object)>

In [57]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [58]:
words = en_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] \xe0\xb0\x90\xe0\xb0\xa1\xe0\xb0\xac\xe0\xb0\x90\xe0\xb0\xaa \xe0\xb0\x86\xe0\xb0\xb0\xe0\xb0\xac\xe0\xb0\x90 \xe0\xb0\xa8\xe0\xb0\x9c\xe0\xb0\xb0 [END]',
       b'[START] \xe0\xb0\xac\xe0\xb0\xaf\xe0\xb0\x82\xe0\xb0\x95\xe0\xb0\x82\xe0\xb0\x97 \xe0\xb0\x9a\xe0\xb0\xab\xe0\xb0\xb2\xe0\xb0\xa4 \xe0\xb0\xa8\xe0\xb0\xa1\xe0\xb1\x81 \xe0\xb0\x9c\xe0\xb0\x9f\xe0\xb0\xb2 \xe0\xb0\xad\xe0\xb0\x9f [END]',
       b'[START] \xe0\xb0\x95\xe0\xb0\xb2\xe0\xb0\x95 \xe0\xb0\xb5\xe0\xb0\x95\xe0\xb0\x9f \xe0\xb0\xa4\xe0\xb0\xb8\xe0\xb0\xa8 \xe0\xb0\x9c\xe0\xb0\xa1\xe0\xb0\x9c . . [END]'],
      dtype=object)>

In [59]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [62]:
token_batch = en_tokenizer.tokenize(en_examples).merge_dims(-2,-1)
words = en_tokenizer.detokenize(token_batch)
words


<tf.RaggedTensor [[b'\xe0\xb0\x90\xe0\xb0\xa1\xe0\xb0\xac\xe0\xb0\x90\xe0\xb0\xaa', b'\xe0\xb0\x86\xe0\xb0\xb0\xe0\xb0\xac\xe0\xb0\x90', b'\xe0\xb0\xa8\xe0\xb0\x9c\xe0\xb0\xb0'], [b'\xe0\xb0\xac\xe0\xb0\xaf\xe0\xb0\x82\xe0\xb0\x95\xe0\xb0\x82\xe0\xb0\x97', b'\xe0\xb0\x9a\xe0\xb0\xab\xe0\xb0\xb2\xe0\xb0\xa4', b'\xe0\xb0\xa8\xe0\xb0\xa1\xe0\xb1\x81', b'\xe0\xb0\x9c\xe0\xb0\x9f\xe0\xb0\xb2', b'\xe0\xb0\xad\xe0\xb0\x9f'], [b'\xe0\xb0\x95\xe0\xb0\xb2\xe0\xb0\x95', b'\xe0\xb0\xb5\xe0\xb0\x95\xe0\xb0\x9f', b'\xe0\xb0\xa4\xe0\xb0\xb8\xe0\xb0\xa8', b'\xe0\xb0\x9c\xe0\xb0\xa1\xe0\xb0\x9c', b'.', b'.']]>

In [63]:
cleanup_text(reserved_tokens, words).numpy()


array([b'\xe0\xb0\x90\xe0\xb0\xa1\xe0\xb0\xac\xe0\xb0\x90\xe0\xb0\xaa \xe0\xb0\x86\xe0\xb0\xb0\xe0\xb0\xac\xe0\xb0\x90 \xe0\xb0\xa8\xe0\xb0\x9c\xe0\xb0\xb0',
       b'\xe0\xb0\xac\xe0\xb0\xaf\xe0\xb0\x82\xe0\xb0\x95\xe0\xb0\x82\xe0\xb0\x97 \xe0\xb0\x9a\xe0\xb0\xab\xe0\xb0\xb2\xe0\xb0\xa4 \xe0\xb0\xa8\xe0\xb0\xa1\xe0\xb1\x81 \xe0\xb0\x9c\xe0\xb0\x9f\xe0\xb0\xb2 \xe0\xb0\xad\xe0\xb0\x9f',
       b'\xe0\xb0\x95\xe0\xb0\xb2\xe0\xb0\x95 \xe0\xb0\xb5\xe0\xb0\x95\xe0\xb0\x9f \xe0\xb0\xa4\xe0\xb0\xb8\xe0\xb0\xa8 \xe0\xb0\x9c\xe0\xb0\xa1\xe0\xb0\x9c . .'],
      dtype=object)

In [64]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

In [65]:
tokenizers = tf.Module()
tokenizers.bd = CustomTokenizer(reserved_tokens, 'pt_vocab.txt')
tokenizers.hd = CustomTokenizer(reserved_tokens, 'en_vocab.txt')

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [66]:
model_name = 'telugu_tokenizer'
tf.saved_model.save(tokenizers, model_name)

INFO:tensorflow:Assets written to: telugu_tokenizer/assets


In [67]:
!zip -r /content/tokenizer.zip /content/telugu_tokenizer


  adding: content/telugu_tokenizer/ (stored 0%)
  adding: content/telugu_tokenizer/assets/ (stored 0%)
  adding: content/telugu_tokenizer/assets/pt_vocab.txt (deflated 72%)
  adding: content/telugu_tokenizer/assets/en_vocab.txt (deflated 70%)
  adding: content/telugu_tokenizer/variables/ (stored 0%)
  adding: content/telugu_tokenizer/variables/variables.data-00000-of-00001 (deflated 69%)
  adding: content/telugu_tokenizer/variables/variables.index (deflated 33%)
  adding: content/telugu_tokenizer/saved_model.pb (deflated 91%)


In [69]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.bd.get_vocab_size().numpy()

7782